In [ ]:
'''Methods check: direct integral, Carr-Madan FFT, Monte Carlo QE, Artificial NN'''

import pandas as pd
import numpy as np
import sys, os, pathlib
import joblib 
import time 


NOTEBOOK_DIR = pathlib.Path(os.getcwd())  
ROOT = NOTEBOOK_DIR.parents[0]  

sys.path.insert(0, str(ROOT))

from src.Heston_integral import heston_integral
from src.Heston_FFT import heston_price_fft
from src.Heston_MC import heston_mc_qe_price
from src.blacksholes import black_scholes
heston_nn = joblib.load("model_ann_3.pkl")

In [2]:
# HESTON PARAMS 
S0   = 105.0
K    = 144.9
m    = K / S0
T    = 1.3
r    = 0.09
q    = 0.0
v0     = 0.075
kappa  = 4.1
theta  = 0.06
sigma_v= 0.4
rho    = -0.3


# FFT PARAMS
alpha  = 1.5
fft_N  = 4096     
eta    = 0.1      

# MC PARAMS
mc_steps = 200
mc_paths = 150_000
seed     = 42

# NN DATASET
X_new = pd.DataFrame([{
    "m": m,
    "T": T,
    "r": r,
    "v0": v0,
    "kappa": kappa,
    "theta": theta,
    "sigma_v": sigma_v,
    "rho": rho
}])

_ = heston_nn.predict(X_new) # warm-up 

In [3]:
# BS
t0 = time.perf_counter()
price_bs = black_scholes(S0, K, T, r, sigma=np.sqrt(v0), option_type="call", q=0.0)
t1 = time.perf_counter()
dt_bs = t1 - t0
print(f"BS  :  {price_bs:.4f}  ({dt_bs*1e3:.2f} ms)")


# INTEGR
t0 = time.perf_counter()
price_int = heston_integral(S0, K, T, r, v0, kappa, theta, sigma_v, rho, option_type="call", q=0.0,
                             int_upper=150.0)
t1 = time.perf_counter()
dt_int = t1 - t0
print(f"INT :  {price_int:.4f}  ({dt_int*1e3:.2f} ms)")


# FFT
t0 = time.perf_counter()
price_fft = heston_price_fft(S0, K, T, r, v0, kappa, theta, sigma_v, rho,
                             option_type="call", q=q, alpha=alpha, N=fft_N, eta=eta)
t1 = time.perf_counter()
dt_fft = t1 - t0
print(f"FFT :  {price_fft:.4f}  ({dt_fft*1e3:.2f} ms)")


# MC
t0 = time.perf_counter()
price_mc, stderr_mc = heston_mc_qe_price(S0, K, T, r, q, v0, kappa, theta, sigma_v, rho,
                                         steps=mc_steps, paths=mc_paths,
                                         antithetic=True, seed=seed, option_type="call")
t1 = time.perf_counter()
dt_mc = t1 - t0
print(f"MC  :  {price_mc:.4f}  ± {1.96*stderr_mc:.6f} (95% CI)  ({dt_mc:.2f} s)")


# NN
t0 = time.perf_counter()
price_nn = float(heston_nn.predict(X_new)[0])
t1 = time.perf_counter()
dt_nn = t1 - t0

print(f"NN  :  {price_nn:.4f}  ({dt_nn*1e3:.3f} ms)")

BS  :  5.5433  (4.61 ms)
INT :  4.1673  (11.01 ms)
FFT :  4.1673  (4.14 ms)
MC  :  4.1398  ± 0.058055 (95% CI)  (1.83 s)
NN  :  4.0161  (0.935 ms)
